In [ ]:
import pandas as pd;
import numpy as np;
import nltk 
import string 
import re
import numpy as np;
import pandas as pd;

!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('movie_reviews') 

import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string
from nltk.corpus import movie_reviews

from google.colab import files
uploaded = files.upload()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [ ]:
# Plan to do the question
# convert the the text to lowercase
# remove all the stop words ,punctuations, numnbers from from the text
# Use lemmatizer on the words
# Tokenizer of the words,
# Pass it on onto the RNN with LSTM

In [ ]:
def to_lowercase(text) :
    return text.lower();
to_lowercase("LLL")

'lll'

In [ ]:
# There can be  two things we can do about the number
# Either remove them or  change them to words
def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result  
input_str = "There are 3 balls in this bag, and 12 in the other one."
remove_numbers(input_str) 

'There are  balls in this bag, and  in the other one.'

In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
lemmatizer = WordNetLemmatizer() 
# making a combined list of stop words and punctuations
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize;
punctuation = list(string.punctuation)
stop_words = stopwords.words('english')
stop_words = stop_words +  punctuation
lemmatizer  = WordNetLemmatizer()
from nltk.corpus import wordnet
def get_simple_pos(tag) :
    if tag.startswith('J') : # everything startring J in pos is Adjective
        return wordnet.ADJ;
    elif(tag.startswith('V')):
        return wordnet.VERB
    elif(tag.startswith('N')):
        return wordnet.NOUN
    elif(tag.startswith('R')):
        return wordnet.ADV
    return wordnet.NOUN;
len(stop_words)

211

In [ ]:
# Using a lemmatizer --> to get to the root word
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
lemmatizer = WordNetLemmatizer() 

# Reading the file and clearing the data

In [ ]:
df = pd.read_csv("training_twitter_x_y_train.csv");
df = df.sample(frac = 1) 
df.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
1375,568930692551221248,negative,United,NaN,thebadvillain,NaN,1,@united do you know how to use twitter bro? Y...,NaN,2015-02-20 16:30:21 -0800,NaN,NaN
3516,569672202037170177,positive,US Airways,NaN,Marcusluft,NaN,0,"@USAirways great, thank you!",NaN,2015-02-22 17:36:51 -0800,"Thunder Bay, ON, Canada",Eastern Time (US & Canada)
5892,568140595580919808,negative,United,NaN,kennykhlee,NaN,0,@united my friend lost luggage on flight 1547....,NaN,2015-02-18 12:10:48 -0800,San Francisco,Pacific Time (US & Canada)
8827,569983990683148288,negative,United,NaN,ousoonerfanatic,NaN,0,@united you are the worst. I will avoid you li...,NaN,2015-02-23 14:15:47 -0800,"Semiahmoo, WA Soonerland",Pacific Time (US & Canada)
419,568564799593820161,negative,United,NaN,Czone12,NaN,0,@united thanks for trying to take my money wit...,"[37.99311597, -84.52114659]",2015-02-19 16:16:26 -0800,lexington KY,NaN


In [ ]:
text = df.text
sentiment = df.airline_sentiment
Y_train = sentiment.values 
information = []
rows = len(df.text);
for i in range(rows) :
    #review_without_numbers = remove_numbers(text.values[i])  # to remove the numbers from the text
    information.append([word_tokenize(text.values[i]),sentiment.values[i]])
len(information)

10980

In [ ]:
from nltk import pos_tag
def clean_review(text):
    output = []
    for word in text :
        if(word.lower() not in stop_words) :
            part_of_speech = pos_tag([word])
            new_word = lemmatizer.lemmatize(word,pos = get_simple_pos(part_of_speech[0][1]))
            output.append(new_word)
    return output;

In [ ]:
clean_information = [[clean_review(review), sentiment] for review, sentiment in information]

In [ ]:
clean_information[0], text.values[0]

([['united',
   'know',
   'use',
   'twitter',
   'bro',
   'terminal',
   'look',
   'like',
   "'s",
   'Bosnia',
   "'s",
   'another',
   'pic',
   'http',
   '//t.co/M9Nywr5Kbs'],
  'negative'],
 "@united do you know how to use twitter bro?  Your terminal looks like it's in Bosnia.  Here's another pic. http://t.co/M9Nywr5Kbs")

In [ ]:
# to make a list of all words,
all_words =[]
set_of_words =set()
for rev, senti in clean_information :
    for words in rev :
      set_of_words.add(words)
      all_words.append(words)

In [ ]:
import nltk
freq = nltk.FreqDist(all_words)
common = freq.most_common(2000) # gives first 2000 most common words
features = [i[0] for i in common]
features[0:10]

['united',
 'flight',
 'AmericanAir',
 'USAirways',
 'SouthwestAir',
 'JetBlue',
 "n't",
 'get',
 "'s",
 'http']

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer
clean_train_reviews = [" ".join(rev) for rev, senti in clean_information]
df_clean_info = pd.DataFrame(clean_train_reviews)

# Getting testing data and cleaning it

In [ ]:
files.upload()
df_test = pd.read_csv("test_twitter_x_test.csv");
df.text
reviews_test=[]
rows = len(df_test.text)
for i in df_test.text :
    reviews_test.append(word_tokenize(i))

clean_test_information = [clean_review(rev) for rev in reviews_test]
clean_test_information = [" ".join(words) for words in clean_test_information]
clean_test_information[0:1]

["AmericanAir car gng DFW Pulled 1hr ago icy road On-hold AA since 1hr Ca n't reach arpt AA2450 Wat 2"]

In [ ]:
# analyzing the clean_info data frame
df_clean_info.head()
df_clean_info["length_of_review"] = 0
count = 0
for i in range(len(df_clean_info[0])) :
  count +=1
  df_clean_info["length_of_review"][i] = len(df_clean_info[0][i].split())
df_clean_info.head()
df_clean_info["length_of_review"].describe()
# looking at this we see  that we have 75 % tweets of length 14

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


count    10980.000000
mean        10.922222
std          4.266485
min          1.000000
25%          8.000000
50%         11.000000
75%         14.000000
max         26.000000
Name: length_of_review, dtype: float64

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
print(len(set_of_words))
max_words = 4000
max_len_sequence = 20 # mean of the tweets
X_train = clean_train_reviews
X_test = clean_test_information
tok = Tokenizer(num_words=max_words) # to get these many maximum number of features
tok.fit_on_texts(X_train) # to get a sequence from the training text
sequences = tok.texts_to_sequences(X_train)  # to apply the fit text we go from  the to tetsing data
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len_sequence, padding = 'post')
#sequences_matrix = sc.fit_transform(sequences_matrix)

14994


In [ ]:
print(sentiment.unique())
number_to_class  =  {
    0 : 'negative',
    1 : 'positive',
    2  : 'neutral'
}
def f(s) :
  if(s=="negative") :
    return 0;
  elif(s=="positive") :
    return 1;
  else : # neutral
    return 2;
Y_train = sentiment.apply(f)
print(Y_train.unique())
Y_train = Y_train.values
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train, 3)
Y_train[0], sentiment[0]

['negative' 'positive' 'neutral']
[0 1 2]


(array([1., 0., 0.], dtype=float32), 'negative')

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(3,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
#model = RNN() the model gave 95 % on testing but 74 % on validation

model = Sequential()
#model.add(SimpleRNN(64, input_shape = (max_len_sequence,1)))
# mask_zero is true vocab = len(set_of_words) + 1
# if not vocab  = len(set_of_words)
model.add(Embedding(len(set_of_words) + 1, 100 , input_length=max_len_sequence, mask_zero = True)) 
model.add(LSTM(256, return_sequences  = True))
# we can add as many lastm layers we want as long as the output is 32 meaning they return sequences = True in prviou layer
model.add(LSTM(256))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))


model.add(Dense(512))
model.add(Activation('relu'))

#optimizer = Adam(learning_rate = 0.1)

model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(sequences_matrix, Y_train,epochs =20, batch_size =  50,validation_split= 0.2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 100)           1499500   
_________________________________________________________________
lstm_8 (LSTM)                (None, 20, 256)           365568    
_________________________________________________________________
lstm_9 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)              

In [ ]:
sequences_matrix[700]

array([ 6, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len_sequence,padding = 'pre')
Y_predict_proba = model.predict(test_sequences_matrix)
Y_predict_proba

array([[9.9929929e-01, 2.0379754e-04, 4.9682340e-04],
       [2.1778284e-01, 5.7257421e-02, 7.2495979e-01],
       [9.9984956e-01, 4.1343425e-05, 1.0907614e-04],
       ...,
       [4.7937050e-01, 1.4136307e-01, 3.7926641e-01],
       [3.0297372e-02, 8.1899238e-01, 1.5071018e-01],
       [8.7060207e-01, 1.6392637e-02, 1.1300531e-01]], dtype=float32)

In [ ]:
import numpy as np
Y_predict = np.argmax(Y_predict_proba, axis=1)
Y_predict


array([0, 2, 0, ..., 0, 1, 0])

In [ ]:
df_pred= pd.DataFrame(Y_predict)
def labels(s) :
  return number_to_class[s];
df_pred[0] = df_pred[0].apply(labels)
df_pred

,0
0,negative
1,neutral
2,negative
3,neutral
4,neutral
...,...
3655,negative
3656,negative
3657,negative
3658,positive


In [ ]:

df_pred.to_csv("LSTMprediction.csv", index = False, header = False)
files.download('LSTMprediction.csv')    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>